In [3]:
import os
import numpy as np
import cv2

In [1]:
#Read in images from folder
DATA_PATH = "/Users/sreeharirammohan/Desktop/Lesions"

classes = os.listdir(DATA_PATH)
del classes[0] #first entry is .DS Store not a directory

print(classes)

X = np.array([])
y = np.array([])

for class_name in classes:
    class_folder = os.path.join(DATA_PATH,class_name)

    sub_files = os.listdir(class_folder)
    
    i = 0
    
    for file in sub_files:
        
        if '.jpg' in file:
            image_filepath = os.path.join(class_folder, file)
            image_as_np_array = cv2.imread(image_filepath,flags=cv2.IMREAD_COLOR)
            
            #decrease the size of the image by 1/4 to 
            image_as_np_array = cv2.resize(image_as_np_array, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
            
            #print(image_as_np_array.shape)

            X = np.append(X, image_as_np_array)
            y = np.append(y, class_name)

            print(i, end='\r', flush=True)

            i += 1
        


NameError: name 'os' is not defined

In [11]:
X = X.reshape(-1,450,600,3)
X.shape

(451, 450, 600, 3)

In [4]:
y.shape

NameError: name 'y' is not defined

In [5]:
#Normalize image array values between 0-1
X /= 255.0

NameError: name 'X' is not defined

In [13]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y)

In [14]:
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1]


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [16]:
X_train.shape

(302, 450, 600, 3)

In [17]:
X_test.shape

(149, 450, 600, 3)

In [18]:
y_train.shape

(302,)

In [19]:
y_test.shape

(149,)

In [20]:
print(y_test)

[0 0 1 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 1 0 0
 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 0 1 0 0 1 0 1 1 0 0 1 0 0 1 1 0 1 1 1 0 1
 0 1 0 0 1 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0
 0 0 0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 1 1 0 0
 1]


In [21]:
import keras 
from keras.models import Sequential
from keras.layers import *

num_classes = 2
input_shape=(450,600,3)
epochs = 5
batch_size = 1

/Users/sreeharirammohan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [22]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test),
          verbose=1)

Train on 302 samples, validate on 149 samples
Epoch 1/5
  3/302 [..............................] - ETA: 25:56:30 - loss: 5.3738 - acc: 0.6667   